In [1]:
%%capture

# libraries for data manipulation
import pandas as pd
import numpy as np

# libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# autoreload modules
%load_ext autoreload
%autoreload 2


# import train_test_split from sklearn
from sklearn.model_selection import train_test_split

# Libraries for Experiment Tracking
import mlflow
import IPython.display

# Libraries for Model Training and Evaluation
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.utils.validation import check_is_fitted

# libraries for file management
import os
import shutil

os.chdir("../src")
import data_utils

os.chdir("../")

In [7]:
os.getcwd()

'c:\\Users\\crist\\Documents\\anyoneai\\final_d\\finalprojectApiAnyoneAI\\training'

In [25]:
df = data_utils.load_dataset(path_rel_dataset= "dataset/raw/PAKDD2010_Modeling_Data.txt",path_rel_variables="dataset/raw/PAKDD2010_VariablesList.XLS")
df.shape

(50000, 53)

In [26]:
IPython.display.HTML(df.head().to_html())

,CLERK_TYPE,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,QUANT_ADDITIONAL_CARDS,POSTAL_ADDRESS_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,EDUCATION_LEVEL,STATE_OF_BIRTH,CITY_OF_BIRTH,NACIONALITY,RESIDENCIAL_STATE,RESIDENCIAL_CITY,RESIDENCIAL_BOROUGH,FLAG_RESIDENCIAL_PHONE,RESIDENCIAL_PHONE_AREA_CODE,RESIDENCE_TYPE,MONTHS_IN_RESIDENCE,FLAG_MOBILE_PHONE,FLAG_EMAIL,PERSONAL_MONTHLY_INCOME,OTHER_INCOMES,FLAG_VISA,FLAG_MASTERCARD,FLAG_DINERS,FLAG_AMERICAN_EXPRESS,FLAG_OTHER_CARDS,QUANT_BANKING_ACCOUNTS,QUANT_SPECIAL_BANKING_ACCOUNTS,PERSONAL_ASSETS_VALUE,QUANT_CARS,COMPANY,PROFESSIONAL_STATE,PROFESSIONAL_CITY,PROFESSIONAL_BOROUGH,FLAG_PROFESSIONAL_PHONE,PROFESSIONAL_PHONE_AREA_CODE,MONTHS_IN_THE_JOB,PROFESSION_CODE,OCCUPATION_TYPE,MATE_PROFESSION_CODE,EDUCATION_LEVEL_2,FLAG_HOME_ADDRESS_DOCUMENT,FLAG_RG,FLAG_CPF,FLAG_INCOME_PROOF,PRODUCT,FLAG_ACSP_RECORD,AGE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3,TARGET_LABEL_BAD=1
ID_CLIENT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,C,5,Web,0,1,F,6,1,0,RN,Assu,1,RN,Santana do Matos,Centro,Y,105,1.0,15.0,N,1,900.0,0.0,1,1,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,9.0,4.0,NaN,NaN,0,0,0,0,1,N,32,595,595,1
2,C,15,Carga,0,1,F,2,0,0,RJ,rio de janeiro,1,RJ,RIO DE JANEIRO,CAMPO GRANDE,Y,20,1.0,1.0,N,1,750.0,0.0,0,0,0,0,0,0,0,0.0,0,Y,,NaN,NaN,N,,0,11.0,4.0,11.0,NaN,0,0,0,0,1,N,34,230,230,1
3,C,5,Web,0,1,F,2,0,0,RN,GARANHUNS,1,RN,Parnamirim,Boa Esperanca,Y,105,1.0,NaN,N,1,500.0,0.0,0,0,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,11.0,NaN,NaN,NaN,0,0,0,0,1,N,27,591,591,0
4,C,20,Web,0,1,F,2,0,0,PE,CABO,1,PE,CABO,PONTE DOS CARVALHOS,N,,NaN,NaN,N,1,500.0,0.0,0,0,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,NaN,NaN,NaN,NaN,0,0,0,0,1,N,61,545,545,0
5,C,10,Web,0,1,M,2,0,0,RJ,RIO DE JANEIRO,1,RJ,Rio de Janeiro,Santa Cruz,Y,20,1.0,12.0,N,1,1200.0,0.0,0,0,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,9.0,5.0,NaN,NaN,0,0,0,0,1,N,48,235,235,1


In [28]:
# split dataset and get stratified target feature
X_train, X_test, y_train, y_test = data_utils.split(
    df, "TARGET_LABEL_BAD=1"
)

In [29]:
X_train.shape

(40000, 52)

In [24]:
def clean_dataset(df):

    cat_features = df.select_dtypes("O").nunique()
    cat_num_features = df.select_dtypes("number").nunique()[
        df.select_dtypes("number").nunique() < 15
    ]

    # create lists with  constant classes, features with +50% empty or NaN values
    cat_constant = list(cat_features[cat_features == 1].index)
    num_constant = ["MONTHS_IN_THE_JOB"]
    catnum_constant = list(cat_num_features[cat_num_features == 1].index)
    catnum_constant_dist = [
        "POSTAL_ADDRESS_TYPE",
        "FLAG_DINERS",
        "FLAG_AMERICAN_EXPRESS",
        "FLAG_OTHER_CARDS",
    ]

    # calculte the half of the dataset
    half_percent_dataset = df.shape[0]/2

    empty_features = list(df.eq(" ").sum()[df.eq(" ").sum() > half_percent_dataset].index)
    nan_features = list(df.isna().sum()[df.isna().sum() > half_percent_dataset].index)

    # grouping lists in only one
    remove_features = [
        *nan_features,
        *empty_features,
        *catnum_constant,
        *catnum_constant_dist,
        *num_constant,
        *cat_constant,
    ]

    # save to pickles 
    path = os.path.join(os.getcwd(), "remove_features.pkl")

    with open(path, "wb") as f:
        pickle.dump(remove_features, f, protocol=pickle.HIGHEST_PROTOCOL)
    # remove selected features from dataframes
    df = df.drop(remove_features, axis=1)
    return df

In [31]:
os.getcwd()

'c:\\Users\\crist\\Documents\\anyoneai\\final_d\\finalprojectApiAnyoneAI\\training'

In [30]:
import pickle

X_train = clean_dataset(X_train)
X_test = clean_dataset(X_test)

In [32]:
X_train.shape

(40000, 32)

In [33]:
X_test.columns

Index(['PAYMENT_DAY', 'APPLICATION_SUBMISSION_TYPE', 'SEX', 'MARITAL_STATUS',
       'QUANT_DEPENDANTS', 'STATE_OF_BIRTH', 'CITY_OF_BIRTH', 'NACIONALITY',
       'RESIDENCIAL_STATE', 'RESIDENCIAL_CITY', 'RESIDENCIAL_BOROUGH',
       'FLAG_RESIDENCIAL_PHONE', 'RESIDENCIAL_PHONE_AREA_CODE',
       'RESIDENCE_TYPE', 'MONTHS_IN_RESIDENCE', 'FLAG_EMAIL',
       'PERSONAL_MONTHLY_INCOME', 'OTHER_INCOMES', 'FLAG_VISA',
       'FLAG_MASTERCARD', 'QUANT_BANKING_ACCOUNTS',
       'QUANT_SPECIAL_BANKING_ACCOUNTS', 'PERSONAL_ASSETS_VALUE', 'QUANT_CARS',
       'COMPANY', 'FLAG_PROFESSIONAL_PHONE', 'PROFESSION_CODE',
       'OCCUPATION_TYPE', 'PRODUCT', 'AGE', 'RESIDENCIAL_ZIP_3',
       'PROFESSIONAL_ZIP_3'],
      dtype='object')

In [ ]:
  # preprocess X_train and X_test features
X_train = data_utils.outliers(X_train)

In [35]:
X_train.shape

(40000, 32)

In [ ]:
X_train, X_test = data_utils.impute_nan(X_train, X_test)

In [ ]:
X_train = data_utils.convert_payment_day(X_train)
X_test = data_utils.convert_payment_day(X_test)

In [ ]:
X_train, X_test = data_utils.encode(X_train, X_test)

C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\crist\AppData\Roami

In [ ]:
X_train = data_utils.convert_age(X_train)
X_test = data_utils.convert_age(X_test)

In [ ]:
X_train, X_test = data_utils.scaling(X_train, X_test)

In [ ]:
IPython.display.HTML(X_train.head().to_html())

In [ ]:
# find nan
X_train.isna().sum().sum()

0

In [ ]:
X_train.shape

(40000, 1777)

In [ ]:
# save X_test, X_test, y_train, y_test to csv files
X_train.to_csv("dataset/processed/X_train.csv", index=False)
X_test.to_csv("dataset/processed/X_test.csv", index=False)
y_train.to_csv("dataset/processed/y_train.csv", index=False)
y_test.to_csv("dataset/processed/y_test.csv", index=False)